In [1]:
# @title Example 2: Query a table with BigQuery DataFrames
# With BigQuery DataFrames, you can use many familiar Pandas methods, but the
# processing happens BigQuery rather than the runtime, allowing you to work with larger
# DataFrames that would otherwise not fit in the runtime memory.
# Learn more here: https://cloud.google.com/python/docs/reference/bigframes/latest

import json
import bigframes.pandas as bf
import pandas as pd
import datetime

bf.options.bigquery.location = "US" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = "hip-principle-416404" #this variable is set based on the dataset you chose to query

In [2]:
df = bf.read_gbq("hip-principle-416404.ecma_github.table_1") #this variable is set based on the dataset you chose to query
df = df[df['payload'].str.contains('"state":"closed"')]
df = df.to_pandas()
df.head()

In [4]:
def addCols(row):
  data = json.loads(row)["issue"]
  return [data["state"], data["created_at"], data["closed_at"], data["labels"]]

def getName(x):
  try:
      return x[4]["name"]
  except:
      return "None"

def parseDF(a):
  a["payload"] = a["payload"].apply(addCols)
  a["state"] = a["payload"].apply(lambda x:  x[0])
  a["created_at"] = a["payload"].apply(lambda x:  x[1])
  a["closed_at"] = a["payload"].apply(lambda x:  x[2])
  a["tags"] = a["payload"].apply(getName)
  a = a.drop(["payload"], axis = 1)
  return a

In [5]:
names = df["name"].unique()
len(names)

614

In [13]:
means = []

for name in names:
  print(name)
  namedf = df[df["name"] == name]
  namedf = parseDF(namedf)

  namedf["opened"] = pd.to_datetime(namedf["created_at"])
  namedf["closed"] = pd.to_datetime(namedf["closed_at"])

  namedf = namedf[namedf["opened"].dt.date >= datetime.date(2015, 1, 1)]

  namedf["timetoclose"] = namedf["closed"] - namedf["opened"]
  bins = namedf.resample('Q', on='opened')["timetoclose"]
  means.append(bins.mean().rename(name).rename_axis(None))

  del namedf
  break

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 9490768f-cde2-4297-a7c9-785489534700
 Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.

In [10]:
meansdf = pd.concat(means, axis=1)
df.head()

In [12]:
meansdf.to_csv("GithubMeans.csv")